In [1]:
from typing import *
import collections
import math
import functools
import heapq
import bisect

# 6257. Delete Greatest Value in Each Row

In [3]:
class Solution:
    def deleteGreatestValue(self, grid: List[List[int]]) -> int:
        ret = 0
        grid = [sorted(g) for g in grid]
        
        for i in range(len(grid[0])):
            maxs = grid[0][i]
            for j in range(len(grid)):
                maxs = max(maxs, grid[j][i])
            ret += maxs
        return ret

In [66]:
class Solution:
    def deleteGreatestValue(self, grid: List[List[int]]) -> int:
        for row in grid:
            row.sort()
        return sum(max(col) for col in zip(*grid))

# 6258. Longest Square Streak in an Array

In [21]:
class Solution:
    def longestSquareStreak(self, nums: List[int]) -> int:
        nums.sort()
        re = collections.defaultdict(int)
        ret = 0
        for i, num in enumerate(nums):
            re[num] = max(re[num], 1)
            re[num ** 2] = max(re[num] + 1, re[num**2])
            ret = max(ret, re[num])
        return ret if ret >= 2 else -1

In [24]:
nums = [2,2]

In [25]:
a = Solution().longestSquareStreak(nums)
a

-1

# 6259. Design Memory Allocator

In [67]:
class RangeModule(object):
    def __init__(self):
        self.ranges = []

    # It also can use bisect
    def _bounds(self, left, right):
        i, j = 0, len(self.ranges) - 1
        for d in (100, 10, 1):
            while i + d - 1 < len(self.ranges) and self.ranges[i+d-1][1] < left:
                i += d
            while j >= d - 1 and self.ranges[j-d+1][0] > right:
                j -= d
        return i, j

    def addRange(self, left, right):
        i, j = self._bounds(left, right)
        if i <= j:
            left = min(left, self.ranges[i][0])
            right = max(right, self.ranges[j][1])
        self.ranges[i:j+1] = [(left, right)]

    def queryRange(self, left, right):
        i = bisect.bisect_left(self.ranges, (left, float('inf')))
        if i: i -= 1
        return (bool(self.ranges) and
                self.ranges[i][0] <= left and
                right <= self.ranges[i][1])

    def removeRange(self, left, right):
        i, j = self._bounds(left, right)
        merge = []
        for k in range(i, j+1):
            if self.ranges[k][0] < left:
                merge.append((self.ranges[k][0], left))
            if right < self.ranges[k][1]:
                merge.append((right, self.ranges[k][1]))
        self.ranges[i:j+1] = merge

In [68]:
class Allocator:

    def __init__(self, n: int):
        self.lens = n
        self.range = RangeModule()
        self.range.addRange(0, n)
        self.re = collections.defaultdict(list)

    def allocate(self, size: int, mID: int) -> int:
        flag = -1
        for l, r in self.range.ranges:
            if r-l >=size:
                self.re[mID].append([l, l+size])
                flag = l
                self.range.removeRange(l, r)
                if r-l >= size:
                    self.range.addRange(l+size, r)
                break
        return flag

    def free(self, mID: int) -> int:
        ret = 0
        for l, r in self.re[mID]:
            self.range.addRange(l, r)
            ret += r-l
        self.re[mID] = []
        return ret

In [29]:
a = [0,12,-1,-1,12,0,0,-1,-1,-1,0,0,0,28,0,0,0,0,12,0,0,-1,0,-1,-1,0,0,0,-1,-1,-1,-1,0,-1,0,0,-1,-1,-1,-1,-1,-1,-1,0,12,0,0]

In [32]:
a[22]

0

# 6260. Maximum Number of Points From Grid Queries

In [65]:

class Solution:
    def maxPoints(self, grid: List[List[int]], queries: List[int]) -> List[int]:
        visited = set()
        leny = len(grid[0])
        q = [[grid[0][0], 0, 0]]
        sortedarr = sorted(queries)
        re = collections.defaultdict(list)
        for i, qu in enumerate(queries):
            re[qu].append(i)
        
        ret = [0] * len(queries)
        
        for n in sortedarr:
            while q and q[0][0] < n:
                _, i, j = heapq.heappop(q)
                if i*leny+j in visied:
                    continue
                visited.add(i*leny + j)
                for nxti, nxtj in [[i-1, j], [i+1, j], [i, j-1], [i, j+1]]:
                    if 0 <= nxti < len(grid) and 0 <= nxtj < len(grid[0]) and nxti * leny + nxtj not in visited:
                        heapq.heappush(q, [grid[nxti][nxtj], nxti, nxtj])
            
            for idx in re[n]:
                ret[idx] = len(visited)
        
        return ret

In [63]:
grid= [[1,2,3],[2,5,7],[3,5,1]]
queries = [5,6,2]

In [64]:
a = Solution().maxPoints(grid, queries)
a

{0} 2
{0, 1, 2, 3, 6} 5
{0, 1, 2, 3, 4, 6, 7, 8} 6


[5, 8, 1]

In [69]:
class Solution:
    def maxPoints(self, grid: List[List[int]], queries: List[int]) -> List[int]:
        m, n = len(grid), len(grid[0])
        mn = m * n

        # 并查集模板
        fa = list(range(mn))
        size = [1] * mn
        def find(x: int) -> int:
            if fa[x] != x:
                fa[x] = find(fa[x])
            return fa[x]
        def merge(from_: int, to: int) -> None:
            from_ = find(from_)
            to = find(to)
            if from_ != to:
                fa[from_] = to
                size[to] += size[from_]

        # 矩阵元素从小到大排序，方便离线
        a = sorted((x, i, j) for i, row in enumerate(grid) for j, x in enumerate(row))

        ans, j = [0] * len(queries), 0
        # 查询的下标按照查询值从小到大排序，方便离线
        for i, q in sorted(enumerate(queries), key=lambda p: p[1]):
            while j < mn and a[j][0] < q:
                _, x, y = a[j]
                # 枚举周围四个格子，值小于 q 才可以合并
                for x2, y2 in (x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1):
                    if 0 <= x2 < m and 0 <= y2 < n and grid[x2][y2] < q:
                        merge(x * n + y, x2 * n + y2)  # 把坐标压缩成一维的编号
                j += 1
            if grid[0][0] < q:
                ans[i] = size[find(0)]  # 左上角的连通块的大小
        return ans